# Transformers
Transforms can be applied to PIL images, tensors, ndarrays, or custom data during creation of the DataSet.  
Complete list of built-in transforms: https://pytorch.org/vision/stable/transforms.html
- On Images: CenterCrop, Grayscale, Pad, RandomAffine, RandomCrop, RandomHorizontalFlip, RandomRotation, Resize, Scale
- On Tensors: LinearTransformation, Normalize, RandomErasing
- Conversion: ToPILImage: from tensor or ndrarray, ToTensor : from numpy.ndarray or PILImage
- Generic: Use Lambda 
- Custom: Write own class
- Compose multiple Transforms: composed = transforms.Compose([Rescale(256), RandomCrop(224)])

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

## Dataset Class

In [2]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        # data loading
        xy = np.loadtxt('data/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_sample = xy.shape[0]
        
        # We do not convert to tensor here
        self.X = xy[:, 1:]
        self.y = xy[:, [0]]
        
        self.transform = transform
    
    def __getitem__(self, index):
        sample = self.X[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    def __len__(self):
        return self.n_sample

## Transform Classes

In [3]:
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [4]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, sample):
        inputs, targets = sample
        return inputs * self.factor, targets

## Single Transform

In [5]:
dataset = WineDataset(transform=ToTensor())

In [6]:
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


## Multiple Transform

In [7]:
# Without Transform
dataset = WineDataset(transform=None)
first_data = dataset[0]
features, labels = first_data
print(features)
print(type(features), type(labels))

[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [8]:
# With Transform
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(10)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(features)
print(type(features), type(labels))

tensor([1.4230e+02, 1.7100e+01, 2.4300e+01, 1.5600e+02, 1.2700e+03, 2.8000e+01,
        3.0600e+01, 2.8000e+00, 2.2900e+01, 5.6400e+01, 1.0400e+01, 3.9200e+01,
        1.0650e+04])
<class 'torch.Tensor'> <class 'torch.Tensor'>
